<a href="https://colab.research.google.com/github/RamadhikaS/STKI/blob/main/KlasteringDokumen_KMeans.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install Sastrawi untuk stemming Bahasa Indonesia
!pip install Sastrawi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 4.8 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.spatial.distance import euclidean

# --- BAGIAN 1: PREPROCESSING (TOKENIZATION & STEMMING) ---

# Buat stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# Data dokumen dari soal
dokumen = {
    'D1': 'PSIS berburu juara Liga Indonesia',
    'D2': 'Hasil putusan Sidang Elit Politik',
    'D3': 'Partai politik berebut suara',
    'D4': 'Manchester United Juara Liga Inggris',
    'D5': 'Timnas Indonesia juara Liga AFC'
}
nama_dokumen = list(dokumen.keys())

# Lakukan stemming pada setiap dokumen
dokumen_stemmed = {}
for nama, teks in dokumen.items():
    dokumen_stemmed[nama] = stemmer.stem(teks)

print("===== Hasil Preprocessing (Stemming) =====")
for nama, teks in dokumen_stemmed.items():
    print(f"{nama}: {teks}")
print("\n" + "="*50 + "\n")


# --- BAGIAN 2: VEKTORISASI (TF-IDF) ---

vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(dokumen_stemmed.values())

# Tampilkan sebagai DataFrame agar mudah dibaca
df_tfidf = pd.DataFrame(
    tfidf_matrix.toarray(),
    index=nama_dokumen,
    columns=vectorizer.get_feature_names_out()
)

print("===== Matriks TF-IDF =====")
print(df_tfidf)
print("\n" + "="*50 + "\n")


# --- BAGIAN 3: ALGORITMA K-MEANS ---

# Inisialisasi: Tentukan centroid awal dari D1 dan D3
centroid_1 = df_tfidf.loc['D1'].values
centroid_2 = df_tfidf.loc['D3'].values

# Lakukan iterasi (biasanya berhenti saat cluster tidak berubah, di sini kita lakukan 2-3 iterasi untuk menunjukkan proses)
max_iterasi = 3
for i in range(max_iterasi):
    print(f"===== ITERASI KE-{i+1} =====")

    # -- Assignment Step --
    # Hitung jarak setiap dokumen ke kedua centroid
    cluster_assignments = {}
    print("\n--- Assignment Step ---")
    for nama, vec in df_tfidf.iterrows():
        jarak_ke_c1 = euclidean(vec, centroid_1)
        jarak_ke_c2 = euclidean(vec, centroid_2)

        # Tentukan cluster berdasarkan jarak terdekat
        if jarak_ke_c1 < jarak_ke_c2:
            cluster_assignments[nama] = 1
        else:
            cluster_assignments[nama] = 2
        print(f"Jarak {nama} ke C1: {jarak_ke_c1:.4f}, ke C2: {jarak_ke_c2:.4f} -> Masuk Cluster {cluster_assignments[nama]}")

    # Ubah hasil assignment menjadi Series untuk kemudahan
    assignments_series = pd.Series(cluster_assignments)

    # -- Update Step --
    # Hitung ulang centroid berdasarkan rata-rata anggota cluster
    print("\n--- Update Step ---")
    vektor_cluster_1 = df_tfidf[assignments_series == 1]
    vektor_cluster_2 = df_tfidf[assignments_series == 2]

    # Pastikan cluster tidak kosong sebelum menghitung mean
    if not vektor_cluster_1.empty:
        centroid_1_baru = vektor_cluster_1.mean().values
    else:
        centroid_1_baru = centroid_1 # Gunakan centroid lama jika kosong

    if not vektor_cluster_2.empty:
        centroid_2_baru = vektor_cluster_2.mean().values
    else:
        centroid_2_baru = centroid_2 # Gunakan centroid lama jika kosong

    print("Centroid 1 baru dihitung dari rata-rata:", list(vektor_cluster_1.index))
    print("Centroid 2 baru dihitung dari rata-rata:", list(vektor_cluster_2.index))

    # Cek konvergensi (apakah centroid berubah?)
    if np.array_equal(centroid_1, centroid_1_baru) and np.array_equal(centroid_2, centroid_2_baru):
        print("\nCentroid tidak berubah. Proses clustering selesai (konvergen).")
        break

    # Perbarui centroid untuk iterasi selanjutnya
    centroid_1 = centroid_1_baru
    centroid_2 = centroid_2_baru
    print("\n" + "="*50 + "\n")

# --- BAGIAN 4: HASIL AKHIR ---
print("===== HASIL AKHIR CLUSTERING =====")
cluster_1_docs = assignments_series[assignments_series == 1].index.tolist()
cluster_2_docs = assignments_series[assignments_series == 2].index.tolist()

print(f"Cluster 1: {cluster_1_docs}")
print(f"Cluster 2: {cluster_2_docs}")

print("\nDetail Anggota Cluster:")
print("\n--- Cluster 1 (Cenderung tentang Olahraga/Liga) ---")
for doc_id in cluster_1_docs:
    print(f"{doc_id}: {dokumen[doc_id]}")

print("\n--- Cluster 2 (Cenderung tentang Politik) ---")
for doc_id in cluster_2_docs:
    print(f"{doc_id}: {dokumen[doc_id]}")

===== Hasil Preprocessing (Stemming) =====
D1: psis buru juara liga indonesia
D2: hasil putus sidang elit politik
D3: partai politik rebut suara
D4: manchester united juara liga inggris
D5: timnas indonesia juara liga afc


===== Matriks TF-IDF =====
         afc      buru      elit     hasil  indonesia   inggris     juara  \
D1  0.000000  0.530899  0.000000  0.000000   0.428326  0.000000  0.355549   
D2  0.000000  0.000000  0.463693  0.463693   0.000000  0.000000  0.000000   
D3  0.000000  0.000000  0.000000  0.000000   0.000000  0.000000  0.000000   
D4  0.000000  0.000000  0.000000  0.000000   0.000000  0.506563  0.339251   
D5  0.530899  0.000000  0.000000  0.000000   0.428326  0.000000  0.355549   

        liga  manchester    partai   politik      psis     putus     rebut  \
D1  0.355549    0.000000  0.000000  0.000000  0.530899  0.000000  0.000000   
D2  0.000000    0.000000  0.000000  0.374105  0.000000  0.463693  0.000000   
D3  0.000000    0.000000  0.523358  0.422242  0.0000